# <center>**Lab Sheet-4**</center>
# <center>**Linear Filters & LMS (Least Mean Squares)**</center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

**1. Create and train a linear regression model using Least Squares.**

In [2]:
X = np.linspace(0, 10, 50)
y = 2*X + 1 + np.random.normal(0, 2, size=X.shape)

# Design matrix
X_mat = np.vstack([X, np.ones_like(X)]).T
# Least Squares solution: w = (X^T X)^-1 X^T y
w = np.linalg.inv(X_mat.T @ X_mat) @ X_mat.T @ y

print("Weights (slope, intercept) =", w)

# Plot regression line
plt.scatter(X, y, label="Data")
plt.plot(X, X_mat @ w, color="red", label="Least Squares Fit")
plt.title("Linear Regression using Least Squares")
plt.legend()
plt.savefig("lab4_least_squares.png", dpi=150); plt.close()

Weights (slope, intercept) = [1.8840332  1.12888618]


**2. Implement the LMS algorithm step-by-step on a time-series dataset.** 

In [3]:
# Generate AR(1) time-series: x_t = 0.8 * x_{t-1} + noise
T = 200
x = np.zeros(T)
for t in range(1,T):
    x[t] = 0.8*x[t-1] + np.random.normal(0,0.5)

# LMS to predict x[t] from x[t-1]
lr = 0.05
epochs = 5
w = 0.0
errors = []

for epoch in range(epochs):
    for t in range(1,T):
        y_pred = w * x[t-1]
        e = x[t] - y_pred
        w += lr * e * x[t-1]
        errors.append(e**2)

print("Final LMS weight =", w)

plt.plot(errors)
plt.title("LMS Error over Time")
plt.xlabel("Iteration")
plt.ylabel("Squared Error")
plt.savefig("lab4_lms_error.png", dpi=150); plt.close()

Final LMS weight = 0.6602643401920488


**3. Plot error surface and gradient descent path during LMS learning.** 

In [4]:
# Error surface: E(w) = mean((x_t - w*x_{t-1})^2)
W = np.linspace(-1,2,100)
E = [np.mean((x[1:] - w*x[:-1])**2) for w in W]

plt.plot(W,E)
plt.title("Error Surface for LMS")
plt.xlabel("Weight")
plt.ylabel("MSE")
plt.savefig("lab4_error_surface.png", dpi=150); plt.close()

# Simulate gradient descent path
w = -1.0
path = []
lr = 0.1
for _ in range(20):
    grad = -2*np.mean((x[1:] - w*x[:-1]) * x[:-1])
    w -= lr*grad
    path.append(w)

plt.plot(W,E,label="Error Surface")
plt.scatter(path, [np.mean((x[1:] - pw*x[:-1])**2) for pw in path],
            color="red", label="GD Path")
plt.title("Gradient Descent Path on Error Surface")
plt.legend()
plt.savefig("lab4_gd_path.png", dpi=150); plt.close()

**4. Compare standard gradient descent vs. learning rate annealing.** 

In [5]:
def train_gd(lr_schedule):
    w = 0.0
    losses = []
    for epoch in range(50):
        lr = lr_schedule(epoch)
        grad = -2*np.mean((x[1:] - w*x[:-1]) * x[:-1])
        w -= lr*grad
        losses.append(np.mean((x[1:] - w*x[:-1])**2))
    return losses

# Constant LR
loss_const = train_gd(lambda e: 0.1)
# Annealing LR
loss_anneal = train_gd(lambda e: 0.1/(1+0.1*e))

plt.plot(loss_const,label="Constant LR")
plt.plot(loss_anneal,label="Annealing LR")
plt.title("GD vs LR Annealing")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend()
plt.savefig("lab4_lr_annealing.png", dpi=150); plt.close()

**5. Analyze the effect of different learning rates on convergence.**

In [6]:
rates = [0.01, 0.1, 0.5]
for r in rates:
    losses = train_gd(lambda e,r=r: r)
    plt.plot(losses,label=f"LR={r}")

plt.title("Effect of Learning Rate on Convergence")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend()
plt.savefig("lab4_learning_rates.png", dpi=150); plt.close()